# Failed models:


Author: Ruiqi Zhong, rz2383@columbia.edu

### In this project we have tried a variety of graphical models, some of them worked and some of them failed. We put the models that work in our Jupiter notebook, and here in this report we discuss the models that fail. Feel free to send email to rz2383@columbia.edu for error details

### PSD model

The original PSD model is defined as follows:
Suppose that there are $K$ population mixtures, $M$ individuals and $N$ SNPs, each genotype has 0, 1, 2 particular allele at a position of DNA. $\alpha, \eta$ are fixed model hyper-parameters.For each $m \in M$,  $\theta_{m} \sim Dir(\alpha)$. For each SNP position and population mixture, $\beta_{nk} \sim Beta(\eta)$. For each of the two allels of a DNA position, we first draw the population mixture $z_{mn0}, z_{mn1} \sim Categorical(\theta_{m})$, then allele is drawn from $w_{mn0}, w_{mn1} \sim \beta_{nz_{mn0}}, \beta_{nz_{mn1}}$. $w_{mn} = w_{mn0} + w_{mn1}$, and $w_{mn}$ is what we observe. The graphical model is shown below:

![](img/PSD.png)

However, unfortunately, Edward does not support inference robustly on LDA type of model, since there are latent discrete variable $z$ in the graphical model. Therefore, we tried to find an alternative to infer the hidden $\theta$. The alternative inference method is highly similar to autoencoder: since two layers of neural network (if given enough hidden units) are able to fit any function, we design the following model:

$z \sim Dir(\alpha), W_{1}, W_{2} \sim Gaussian(0, \Sigma), hidden_1 = relu(W_{1}z), w_{out} = Bernoulli(\sigma(W_{2} hidden_1)) + Bernoulli(\sigma(W2z))$

To capture the most important information about an individual, the hidden $z_{m}$ will be forced to correpsond to $\theta_{m}$ and thus learn the hidden $\theta$. To infer $\beta_{k}$, simply activate the $k^{th}$ dimension of $z$, setting all others to 0 and infer $z$ according to $w_{out}$. Since there are no hidden discrete varible, we thought that the inference will not have any problem. Unfortunately, it still does not work (see failed PSD for detail). Even for a small toy data set, inference method KLqp is fragile and frequently gives out nan. Even if when it works, the $\theta_{m}$ it gets cannot get far away from $(\frac{1}{K},\frac{1}{K}...)$. HMC does not work either, since it seems to have trouble recognizing the correct dimension of hidden Dirichlet variable. 

![](img/ApproximatePSD.png)

### Probabilistic "PCA" with sigmoid activation

We also tried other dimension reduction model on MNIST data set besides bayesian autoencoder. In particular, we basically followed the tutorials from here: http://edwardlib.org/tutorials/probabilistic-pca. The graphical model is as follows:

$z \sim Gaussian(0, \Sigma_{1}), W \sim Gaussian(0, \Sigma_{2}), x \sim Bernoulli(\sigma(Wz))$, where $x$ is the MNIST image we observe.

However, because the data set is too large, the process is always killed during the process. We also tried to use batch training; however, for the inference update we need current $qz$, $qW$ and corresponding $x$. However, since $qz$ is a tensor type (in particular, random variable type), it cannot be batched (placeholder only takes in data, not tensors) for training update. Therefore, this model also failed. 

![](img/PPCA.png)